In [1]:
## Text to embeddings
## embeddings available for openai and ollama 

In [2]:
from mb_rag.rag.embeddings import embedding_generator

In [3]:
embedding_generator?

Init signature:
embedding_generator(
    model: str = 'openai',
    model_type: str = 'text-embedding-3-small',
    vector_store_type: str = 'chroma',
    logger=None,
    model_kwargs: dict = None,
    vector_store_kwargs: dict = None,
) -> None
Docstring:     
Class to generate embeddings for the RAG model abnd chat with data
Args:
    model: type of model. Default is openai. Options are openai, anthropic, google, ollama
    model_type: type of model. Default is text-embedding-3-small. Options are text-embedding-3-small, text-embedding-3-large, text-embedding-ada-002 for openai.
    vector_store_type: type of vector store. Default is chroma
    logger: logger
    model_kwargs: additional arguments for the model
    vector_store_kwargs: additional arguments for the vector store
File:           ~/Desktop/mb_packages/mb_rag/mb_rag/rag/embeddings.py
Type:           type
Subclasses:     

In [4]:
em_gen = embedding_generator(model="openai",model_type="text-embedding-3-small",vector_store_type="chroma")

/home/malav/Desktop/mb_packages/mb_rag/mb_rag/rag/embeddings.py:181: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  vector_store = Chroma()


In [5]:
em_gen.generate_text_embeddings(text_data_path=['./examples/book_data.txt'],chunk_size=500,chunk_overlap=5,
                                folder_save_path='./examples/book_data_embeddings',replace_existing=True)

Created a chunk of size 510, which is longer than the specified 500
Created a chunk of size 3416, which is longer than the specified 500
Created a chunk of size 553, which is longer than the specified 500
Created a chunk of size 888, which is longer than the specified 500
Created a chunk of size 753, which is longer than the specified 500
Created a chunk of size 542, which is longer than the specified 500
Created a chunk of size 1093, which is longer than the specified 500
Created a chunk of size 1748, which is longer than the specified 500
Created a chunk of size 729, which is longer than the specified 500
Created a chunk of size 693, which is longer than the specified 500
Created a chunk of size 714, which is longer than the specified 500
Created a chunk of size 804, which is longer than the specified 500
Created a chunk of size 1922, which is longer than the specified 500
Created a chunk of size 544, which is longer than the specified 500
Created a chunk of size 533, which is longer

In [5]:
em_loading = em_gen.load_embeddings(embeddings_folder_path='./examples/book_data_embeddings')

In [6]:
em_loading

In [7]:
em_retriever = em_gen.load_retriever(embeddings_folder_path='./examples/book_data_embeddings',search_params=[{"k": 2,"score_threshold": 0.1}])

In [8]:
ans = em_retriever.invoke("what is the Book about?")
# try:
    # ans = em_retriever.invoke("what is the Book about?")
# except AttributeError:
    # If ChatOpenAI doesn't have embed_query, use the embedding model directly
# query_embedding = em_gen.model.embed_documents(["what is the Book about?"])[0]
# ans = em_retriever.get_relevant_documents(query_embedding)

In [9]:
for i,k in enumerate(ans):
    print(f"Document {i}:\n{k.page_content}\n")
    print(f"Source: {k.metadata['source']}\n")

Document 0:
This table had a cover made out of beautiful oilcloth, with a red and
blue spread-eagle painted on it, and a painted border all around. It
come all the way from Philadelphia, they said. There was some books,
too, piled up perfectly exact, on each corner of the table. One was a
big family Bible full of pictures. One was Pilgrim’s Progress, about a
man that left his family, it didn’t say why. I read considerable in it
now and then. The statements was interesting, but tough. Another was
Friendship’s Offering, full of beautiful stuff and poetry; but I didn’t
read the poetry. Another was Henry Clay’s Speeches, and another was Dr.
Gunn’s Family Medicine, which told you all about what to do if a body
was sick or dead. There was a hymn book, and a lot of other books. And
there was nice split-bottom chairs, and perfectly sound, too—not bagged
down in the middle and busted, like an old basket.

Source: ./examples/book_data.txt



In [10]:
## testing history aware rag chain / retriever
# Generate RAG chain
rag_chain = em_gen.generate_rag_chain(retriever=em_retriever)


In [11]:
# Test the RAG chain
test_questions = [
    "What is the main theme of the book?",
    "Who are the main characters?",
    "Can you summarize the plot?",
    "What is the setting of the story?",
    "What genre does this book belong to?"
]

print("Testing RAG chain with multiple questions:\n")
for question in test_questions:
    print(f"Question: {question}")
    response = em_gen.conversation_chain(question, rag_chain,file='./examples/book_data_rag_chain.txt')
    print(f"Response: {response}\n")

Testing RAG chain with multiple questions:

Question: What is the main theme of the book?
Response: The main theme of "Adventures of Huckleberry Finn" is the conflict between civilization and freedom. It explores the struggles of the protagonist, Huck, as he seeks to escape societal expectations and constraints, while also addressing issues of race and identity in pre-Civil War America.
Saved file : ./examples/book_data_rag_chain.txt
Response: ["content='You : What is the main theme of the book?'", 'content="I\'m sorry, but I need more context about the book you\'re referring to in order to provide an answer. Could you please specify the title of the book?"', "content='You : Who are the main characters?'", 'content="I\'m sorry, but I need more context about the book you\'re referring to in order to provide an answer. Could you please specify the title of the book?"', "content='You : Can you summarize the plot?'", 'content=\'The plot follows two boys, Huck Finn and Tom Sawyer, who find 

In [12]:
# Test query_embeddings function
print("Testing query_embeddings function:\n")
query = "What are the key themes explored in the book?"
result = em_gen.query_embeddings(query)
print(f"Query: {query}")
print(f"Result: {result}\n")

# Test get_relevant_documents function
print("Testing get_relevant_documents function:\n")
query = "Describe the main conflict in the story."
docs = em_gen.get_relevant_documents(query)
print(f"Query: {query}")
print("Relevant documents:")
for i, doc in enumerate(docs):
    print(f"Document {i+1}:")
    print(f"Content: {doc.page_content[:100]}...")  # Print first 100 characters
    print(f"Source: {doc.metadata['source']}\n")


Testing query_embeddings function:



No relevant docs were retrieved using the relevance score threshold 0.1
/home/malav/Desktop/mb_packages/mb_rag/mb_rag/rag/embeddings.py:258: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  return self.retriever.get_relevant_documents(query)


Query: What are the key themes explored in the book?
Result: []

Testing get_relevant_documents function:

Query: Describe the main conflict in the story.
Relevant documents:
Document 1:
Content: But it warn’t. It was Jack-o’-lanterns, or lightning bugs; so he set
down again, and went to watchin...
Source: ./examples/book_data.txt



In [15]:
em_gen.load_conversation('./examples/book_data_rag_chain.txt',list_type=True)

["content='You : What is the main theme of the book?'",
 'content=\'The main theme of "Adventures of Huckleberry Finn" is the conflict between civilization and freedom. It explores the struggles of the protagonist, Huck, as he seeks to escape societal expectations and constraints, while also addressing issues of race and identity in pre-Civil War America.\'',
 "content='You : Who are the main characters?'",
 'content=\'The main characters in "Adventures of Huckleberry Finn" are Huck Finn and Tom Sawyer. Huck Finn is the protagonist, and Tom Sawyer is his friend and comrade in various adventures.\'',
 "content='You : Can you summarize the plot?'",
 'content=\'"Adventures of Huckleberry Finn" follows Huck Finn, a young boy who fakes his own death to escape his abusive father. He teams up with Jim, a runaway slave, and they journey down the Mississippi River on a raft, encountering various adventures and characters. Throughout their journey, Huck grapples with his conscience and societal 